In [ ]:
import pandas as pd

df = pd.read_csv("../../data/processed/model_df_vote_classification_normalized.csv")

In [ ]:
class_counts = df["voted_classified"].value_counts()
print("Rodzaj danej odpowiedzi i jej ilość:")
print(class_counts)

class_percent = class_counts / len(df) * 100
print("\nRodzaj danej odpowiedzi i jej ilość procentowa:")
print(class_percent.round(2))

In [ ]:
df["is_correct"] = df["voted_classified"].isin(["Correct Answers", "Best Answer"])

accuracy_by_model = df.groupby("model")["is_correct"].mean()*100
hallucination_rate = 100 - accuracy_by_model

print("Precyzja danego modelu procentowo:")
print(accuracy_by_model.round(2))
print("\nIlość halucynacji danego modelu procentowo:")
print(hallucination_rate.round(2))

In [ ]:
acc_by_lorem = (df.groupby("lorem_length")["is_correct"].mean() * 100).round(2)
print("Procentowa ilość poprawnych odpowiedzi w zależności od ilości znaków lorem ipsum:")
print(acc_by_lorem)

In [ ]:
acc_by_prompt = (df.groupby("prompt_type")["is_correct"].mean() * 100).round(2)
print("Procentowa ilość poprawnych odpowiedzi w zależności od podania danej odpowiedzi:")
print(acc_by_prompt)

In [ ]:
import seaborn as sns, matplotlib.pyplot as plt

sns.barplot(x=accuracy_by_model.index, y=accuracy_by_model.values)
plt.title("Ilość poprawnych odpowiedzi procentowo udzielonych przez dany model")
plt.ylabel("Ilość poprawnych odpowiedzi %")
plt.show()

heat_data = df.pivot_table(index="model", columns="lorem_length",
                           values="is_correct", aggfunc="mean")
sns.heatmap(heat_data, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title("Ilość poprawnych odpowiedzi danego modelu vs długość lorem ipsum")
plt.show()

In [ ]:
import numpy as np

pivot_acc_prompt = df.pivot_table(index="model",
                           columns="prompt_type",
                           values="is_correct",
                           aggfunc=np.mean)
print(pivot_acc_.round(3))

In [ ]:
import seaborn as sns, matplotlib.pyplot as plt

sns.barplot(x=acc_by_prompt.index, y=acc_by_prompt.values,
            palette="viridis")
plt.title("Ilość poprawnych odpowiedzi wg podanego typu odpowiedzi")
plt.ylabel("Średnia dokładność")
plt.xlabel("Podany typ odpowiedzi")
plt.xticks(rotation=30)
plt.show()

In [ ]:
stack = pd.crosstab(df["prompt_type"], df["voted_classified"])
stack_norm = stack.div(stack.sum(axis=1), axis=0)

stack_norm.plot(kind="bar", stacked=True, colormap="Set3")
plt.title("Rozkład rodzaju odpowiedzi wg podanego typu odpowiedzi")
plt.ylabel("Udział procentowy")
plt.xlabel("Typ podanej odpowiedzi")
plt.xticks(rotation=30)
plt.show()

In [ ]:
sns.heatmap(pivot_acc_prompt, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title("Ilość poprawnych odpowiedzi danego modelu vs rodzaj podanej odpowiedzi")
plt.ylabel("Model")
plt.xlabel("Typ podanej odpowiedzi")
plt.xticks(rotation=30)
plt.show()

In [ ]:
import numpy as np

pivot_acc_lorem = df.pivot_table(index="model",
                           columns="lorem_length",
                           values="is_correct",
                           aggfunc=np.mean)
print(pivot_acc_lorem.round(3))